In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import requests
from bs4 import BeautifulSoup

In [ ]:
def movie_data(url):

    movie_names= []
    run_time=[]
    imdb_rating=[]
    metascore_rating=[]
    genre = []
    total_reviews = []
    movie_collection = []
    
    
    driver = webdriver.Chrome("C:\\Users\\Aditya\\Desktop\\chromedriver.exe")
    driver.get(url)
    for _ in range(9):    ### Run in on 9 different pages
        
        soup = BeautifulSoup(driver.page_source,"html.parser")
        movie_containers = soup.find_all('div', class_='lister-item mode-advanced')
        for containers in movie_containers:
            if containers.find('div', class_ = 'ratings-metascore') is not None:
                
              
                    movie = containers.find('h3').find('a').text   ##movie name
                    movie_names.append(movie)
    
                    runtime = containers.find(class_='runtime').text   ### runtime 
                    run_time.append(runtime)
    
                    imdb = containers.find('strong').text    ### imdb rating
                    imdb_rating.append(imdb)
    
                    metascore = containers.find(class_='metascore').text   ## Metascore rating
                    metascore_rating.append(metascore)
        
                    genre_class = containers.find(class_='genre').text.split()   ### genre
                    genre.append(genre_class)  
     
                    reviews = containers.find(class_='sort-num_votes-visible').find_all('span')[1].text ### total_reviews
                    total_reviews.append(reviews)
            
                
            
    
        time.sleep(6)
        test = driver.find_element_by_class_name('lister-page-next')   ###click on next for getting more movies
        test.click()
    
    df = pd.DataFrame({'Movie': movie_names,
                        'Run_time': run_time,
                       'IMDB': imdb_rating,
                     'Metascore': metascore_rating,
                       'Genre' : genre,
                       'Total_reviews' : total_reviews
                       }) 
    return(df)
    

### Collecting Data of different genre

In [ ]:
df_movie_info_scifi = movie_data('https://www.imdb.com/search/title?genres=sci_fi&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=5aab685f-35eb-40f3-95f7-c53f09d542c3&pf_rd_r=9KXP1M3CEWMQEN9REPFJ&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&page=1&ref_=adv_prv')
df_movie_fantasy = movie_data('https://www.imdb.com/search/title?genres=fantasy&explore=title_type,genres&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=fd0c0dd4-de47-4168-baa8-239e02fd9ee7&pf_rd_r=8NG1K4YX04G1YFQ3MEAH&pf_rd_s=center-4&pf_rd_t=15051&pf_rd_i=genre&ref_=ft_gnr_pr4_i_3')
df_movie_animation = movie_data('https://www.imdb.com/search/title?genres=animation&explore=title_type,genres&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=fd0c0dd4-de47-4168-baa8-239e02fd9ee7&pf_rd_r=8NG1K4YX04G1YFQ3MEAH&pf_rd_s=center-4&pf_rd_t=15051&pf_rd_i=genre&ref_=ft_gnr_pr4_i_1')
df_movie_romance = movie_data('https://www.imdb.com/search/title?genres=romance&explore=title_type,genres&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=e0da8c98-35e8-4ebd-8e86-e7d39c92730c&pf_rd_r=8NG1K4YX04G1YFQ3MEAH&pf_rd_s=center-2&pf_rd_t=15051&pf_rd_i=genre&ref_=ft_gnr_pr2_i_1')
df_movie_mystery = movie_data("https://www.imdb.com/search/title?genres=mystery&explore=title_type,genres&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=f1cf7b98-03fb-4a83-95f3-d833fdba0471&pf_rd_r=8NG1K4YX04G1YFQ3MEAH&pf_rd_s=center-3&pf_rd_t=15051&pf_rd_i=genre&ref_=ft_gnr_pr3_i_2")
df_movie_action_comedy = movie_data("https://www.imdb.com/search/title?genres=action,comedy&explore=title_type,genres&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=15aedf0d-e467-4a1a-922a-e0cbd7299e84&pf_rd_r=8NG1K4YX04G1YFQ3MEAH&pf_rd_s=center-5&pf_rd_t=15051&pf_rd_i=genre&ref_=ft_gnr_pr5_i_2")

In [ ]:
total_df = pd.concat([df_movie_info_scifi, df_movie_fantasy, df_movie_animation, df_movie_romance, df_movie_romance, df_movie_mystery, df_movie_action_comedy], ignore_index = True)

In [ ]:
total_df.to_pickle("Totaldata.pkl")  ## pickling the data

### Scraper for collecting comments

In [ ]:
def scrape_movie_comments(url):
    comments = []
    total_df = pd.DataFrame()
    driver = webdriver.Chrome("C:\\Users\\Aditya\\Desktop\\chromedriver.exe")
    driver.get(url)
    soup = BeautifulSoup(driver.page_source,"lxml")
    movie_list =[]
    data = soup.find_all('h3',attrs={'class':"lister-item-header"})
    for div in data:
        links = div.find_all('a')
        for a in links:
            movie_list.append(a.text)
    movie_list = (['"{0}"'.format(y) for y in movie_list])
    time.sleep(3)
    for x in range(7):  ## 6 more pages loaded 
       
        driver.get(url)
        test = driver.find_element_by_xpath('//a[text()={}]'.format(movie_list[x])) ### Clicks a movie title
        time.sleep(3)
        test.click()
        soup = BeautifulSoup(driver.page_source,"html.parser")
        movie_containers = soup.find_all(class_ ='subText')
        review_number = movie_containers[1].find('a').text
        ### finding the total reviews section to click 
        test = driver.find_element_by_xpath('//a[text()="See all {} reviews"]'.format(review_number)) 
        time.sleep(3)
        test.click()
        
        for _ in range(4):   ### Loads the review page 4 more times
            test = driver.find_element_by_class_name('ipl-load-more__button')
            time.sleep(3)
            test.click()
            
        soup = BeautifulSoup(driver.page_source,"html.parser")
       
        for element in soup.find_all(class_='title'):  ### collects all the comment headers
             comments.append(element.text)
          
        df = pd.DataFrame({'Comments': comments,
                        'Movie_name': movie_list[x]})
        total_df = total_df.append(df)
        
    return(total_df)